In [1]:
!pip install langchain sentence-transformers langchain-community chromadb torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 959.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 489.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 181.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 186.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 184.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 204.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 213.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 244.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 236.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 328.4 

In [2]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.3 MB/s eta 0:00:00


In [16]:
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00


In [5]:
import json
from langchain_text_splitters import RecursiveJsonSplitter

In [6]:
doc_path = '/content/summarized.json'
json_content = json.load(open(doc_path))

In [7]:
splitter = RecursiveJsonSplitter(max_chunk_size=1000)
docs = splitter.create_documents(texts=json_content)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-8-e39c175a87a6>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create 

In [9]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings_model)

In [10]:
import os
key = "<groq_api_key>"
os.environ["GROQ_API_KEY"] = key

In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=200,
    timeout=None,
    max_retries=2,
)

In [20]:
from langchain_core.prompts import ChatPromptTemplate
inference_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that Answers questions ONLY using the context provided in ONE WORD. Think STEP by STEP, use only the RELEVANT context. Do NOT Explain. you will be tipped if you follow instructions.",
        ),
        ("human", "Context: {context} \n\n Question: {input} \n\n Answer:"),
    ]
)

comparison_template = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a helpful assistant that accepts context along with an comparision question. The answer to the question should be a SINGLE word, YES OR NO. ONLY Use the context given to generate the answer. NO EXPLANATION. you will be tipped if you follow instructions.",
        ),
        ("human", "Context: {context} \n\n Question: {input} \n\n Answer:"),
])

temporal_template = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a helpful assistant that accepts context along with an temporal question. The answer to the question should be a SINGLE word, YES OR NO. ONLY use the context given to generate the answer. NO EXPLANATION, you will be tipped if you follow instructions.",
        ),
        ("human", "Context: {context} \n\n Question: {input} \n\n Answer:"),
])

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain_inf = create_stuff_documents_chain(llm, inference_template)
doc_chain_cmp = create_stuff_documents_chain(llm, comparison_template)
doc_chain_tmp = create_stuff_documents_chain(llm, temporal_template)

In [22]:
from langchain.chains import create_retrieval_chain
retriever = db.as_retriever(search_kwargs={"k":3})

retriever_chain_inf = create_retrieval_chain(retriever, doc_chain_inf)
retriever_chain_cmp = create_retrieval_chain(retriever, doc_chain_cmp)
retriever_chain_tmp = create_retrieval_chain(retriever, doc_chain_tmp)

In [15]:
def classify_query(query):
    query = query.casefold()

    #remove non alphabet characters
    query = ''.join(e for e in query if e.isalnum() or e.isspace())

    comparison_keywords = ["does", "do", "are"]
    if query.startswith(tuple(comparison_keywords)):
        return "comparison_query"

    temporal_keywords = ["between", "after", "has", "was", "is", "before", "considering", "regarding", "according"]
    if query.startswith(tuple(temporal_keywords)):
        return "temporal_query"

    inference_keywords = ["which", "who", "what", "considering", "in"]
    if query.startswith(tuple(inference_keywords)):
        return "inference_query"

    confusing = ['did']
    if query.startswith(tuple(confusing)):
        if any(keyword in query for keyword in ["disagree with", "identically", 'both', 'while', 'also', 'similarly', "as the", "in the same manner that", "similar", "than", "align with", "does the same", "compared", "contrast", "unlike", "contrast"]):
            return "comparison_query"
        else:
            return "temporal_query"

    return "Unclassified"

In [16]:
import ast

In [17]:
def get_answer(ques):
  answer_dict = dict()

  answer_dict['query'] = ques
  answer_dict['question_type'] = classify_query(ques)

  if answer_dict['question_type'] == "comparison_query":
    response = retriever_chain_cmp.invoke({"input": ques})
  elif answer_dict['question_type'] == "temporal_query":
    response = retriever_chain_tmp.invoke({"input": ques})
  else:
    response = retriever_chain_inf.invoke({"input": ques})

  answer_dict['answer'] = response['answer']
  answer_dict['evidence_list'] = [ast.literal_eval(doc.page_content) for doc in response['context']]

  print(response)
  return answer_dict

In [1]:
import anvil.server

anvil.server.connect("<anvil_uplink_key>")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [18]:
@anvil.server.callable
def get_final_result(query):
  temp = get_answer(query)
  with open('answer.json', 'w') as file:
    file.write(json.dumps(temp, indent=4))
  return temp['answer'], temp['question_type'], json.dumps(temp, indent=4)

In [ ]:
anvil.server.wait_forever()

{'input': "Does 'The Independent - Life and Style' article suggesting Prince William's emotional state regarding Princess Diana's death align with the same publication's depiction of the events leading up to her death in 'The Crown season six'?", 'context': [Document(metadata={}, page_content='{"title": "Fans react to most \\u2018heartbreaking\\u2019 scene in The Crown", "author": "Meredith Clark", "source": "The Independent - Life and Style", "published_at": "2023-11-17T09:35:32+00:00", "category": "entertainment", "url": "https://www.independent.co.uk/life-style/royal-family/news/princess-diana-st-tropez-b2449049.html", "fact": "The first four episodes of The Crown season six are officially here. Many viewers believe one scene has depicted the most devastating moment. In episode three, Diana\\u2019s partner Dodi Fayed surprises her with a quick trip to Paris. The princess is keen to return home to the United Kingdom, and is eager to speak to her sons over the phone."}'), Document(met